In [1]:
import numpy,csv,os,re

In [ ]:
import csv #导出CSV文件（点位+路段）信息
section_dict={}
info=[0,2,6,8,24,44,45]
with open(r'C:\Users\e-wel\Documents\人工智慧\交通数据\路段信息.csv','r',errors='ignore',encoding='utf-8-sig') as file:
    rows=csv.reader(file)
    for index,row in enumerate(rows):
        if index>0:
            section_id=row[1]
            section_dict[section_id]=[row[each] for each in info]
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\40天统计信息及路段情况\路段信息.npy',section_dict)

In [ ]:
rainy_dates=[]
with open(r'C:\Users\e-wel\Desktop\2018.5.txt','r',errors='ignore',encoding='utf-8') as status_file:
    line_list=status_file.readlines()
    for line in line_list:
        raw_record=line.split(' ')[:2]
        obj=re.search('(\d*)月(\d*)日',raw_record[0]).group(1,2)
        date=obj[0]+obj[1]
        if ('大雨' or '中雨') in raw_record[1]:
            rainy_dates.append(date)

print(rainy_dates) 

In [ ]:
kakou=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天统计信息及路段情况\卡口点位信息.npy',allow_pickle=True).item()

In [ ]:
area_dict={}
for kk,value in kakou.items():
    if value[2] not in area_dict:
        area_dict[value[2]]=[]
    area_dict[value[2]].append([kk,value[1]])

In [ ]:
matched={}#点位路段匹配
for seg,value in segment.items():
# value =segment['35060039']
    dianwei_area=area_dict[value[5]]
    location=value[4].split(',')
    location=[float(each) for each in location]
    weidus=[]
    jindus=[]
    i=0
    while i < len(location)-1:
        jindus.append(location[i])
        weidus.append(location[i+1])
        i+=2
    weidu_max=max(weidus)
    jindu_max=max(jindus)
    weidu_min=min(weidus)
    jindu_min=min(jindus)
    dianwei_list=[]
    for dianwei in dianwei_area:
        j,w=dianwei[1]
        j=float(j)
        w=float(w)
        if j<jindu_max and j>jindu_min and w<weidu_max and w>weidu_min:
            # matched[dianwei[0]]=seg
            dianwei_list.append(dianwei[0])
    segment[seg].append(dianwei_list)
            

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\点位_to_路段.npy',matched)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\区域_to_点位.npy',area_dict)

In [ ]:
for seg,value in matched.items():
    if len(value)>1:
        print(seg,len(value))

In [ ]:
time_stamp=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\time_stamp.npy',allow_pickle=True).item()

In [ ]:
section_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\路段信息.npy',allow_pickle=True).item()

In [ ]:
matched=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\点位_to_路段.npy',allow_pickle=True).item()

In [ ]:
def dump_out(date,txt_name):
    save_root=r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录'
    date_dict={}#section_id:{timestamp_0:value}
    with open(r'C:\Users\e-wel\Documents\人工智慧\交通数据\2018.08-09卡口状态数据\{}'.format(txt_name),'r',errors='ignore',encoding='utf-8') as status_file:
        line_list=status_file.readlines()
        for line in line_list:
            raw_record=line.split(',')
            section_id=int(section_dict[raw_record[1]][0])
            if section_id not in date_dict:
                date_dict[section_id]=time_stamp.copy()
            time_stamp_str=re.search('(\d*:\d*):\d*',raw_record[4]).group(1)
            hour=int(time_stamp_str[0:2])
            if hour<23 and hour>7:
                date_dict[section_id][time_stamp_str]=raw_record[3]
    save_path=os.path.join(save_root,date,date+'_raw_data.npy')
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    numpy.save(save_path,date_dict)        


In [ ]:
def dump_out_flow(date,txt_name):
    save_root=r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录'
    date_dict={}#section_id:{timestamp_0:value}
    with open(r'C:\Users\e-wel\Documents\人工智慧\交通数据\2018.08-09卡口流量数据\5min\{}'.format(txt_name),'r',errors='ignore',encoding='utf-8') as status_file:
        line_list=status_file.readlines()
        for line in line_list:
            raw_record=line.split(',')
            dianwei_id=raw_record[0]
            if dianwei_id not in matched:
                continue
            section_id=matched[dianwei_id]
            if section_id not in date_dict:
                date_dict[section_id]={}
            time_stamp_str=re.search('(\d*:\d*):\d*',raw_record[5]).group(1)
            hour=int(time_stamp_str[0:2])
            if hour<23 and hour>7:
                if time_stamp_str not in date_dict[section_id]:
                    date_dict[section_id][time_stamp_str]=[[],[]]
                date_dict[section_id][time_stamp_str][0].append(raw_record[4])
                date_dict[section_id][time_stamp_str][1].append(raw_record[2])
    save_path=os.path.join(save_root,date,date+'_flow.npy')
    if not os.path.exists(os.path.dirname(save_path)):
        os.makedirs(os.path.dirname(save_path))
    numpy.save(save_path,date_dict)        


In [ ]:
dump_out('0603','SECTION_STATUS_COMPUTE_20180603_033229.txt')

In [ ]:
for file in os.listdir(r'C:\Users\e-wel\Documents\人工智慧\交通数据\2018.08-09卡口流量数据\5min')[1:]:
    obj=re.search('2018(\d*)',file)
    if obj is not None:
        date=obj.group(1)
        dump_out_flow(date,file)
        print(date)

In [ ]:
new_dates=['0501','0502','0503','0504','0506','0505','0507','0513','0514','0520','0521','0604','0527','0528','0610','0611','0617','0618','0624','0625']

In [ ]:
save_root=r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录'
for date in new_dates:
    save_path=os.path.join(save_root,date,date+'_raw_data.npy')
    # new_dict={}
    old_dict=numpy.load(save_path,allow_pickle=True).item()
    for key,value in old_dict.items():
        ts_copy=time_stamp.copy()
        ts_copy.update(value)
        old_dict[key]=ts_copy
    numpy.save(save_path,old_dict)
    print(date)


In [ ]:
save_root=r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录'
segment_dict={}
for date in new_dates:
    save_path=os.path.join(save_root,date,date+'_raw_data.npy')
    old_dict=numpy.load(save_path,allow_pickle=True).item()
    for seg,date_record in old_dict.items():
        if seg not in segment_dict:
            segment_dict[seg]=[]
        segment_dict[seg].append(list(date_record.values()))
numpy.save('补充20天原始记录汇总.npy',segment_dict)

In [ ]:
d1=numpy.load('补充20天原始记录汇总.npy',allow_pickle=True).item()
new_dict={}
for seg,value in d1.items():
    new_dict[seg]=[]
    for day_record in value:
        for index in range(900):
            if day_record[index]=='':
                day_record[index]=0
            else:
                day_record[index]=int(day_record[index])
        new_dict[seg].append(day_record)

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充20天记录汇总_整数.npy',new_dict)

In [ ]:
new_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充20天记录汇总_整数.npy',allow_pickle=True).item()

In [5]:
min_tt=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天统计信息及路段情况\min_travel_time.npy',allow_pickle=True).item()

In [ ]:
type(min_tt['1'])

In [ ]:
for seg,records in new_dict.items():
    for day_index,day in enumerate(records):
        for index,each in enumerate(day):
            if each==0:
                if index==0:
                    day[0]=min_tt[str(seg)]
                else:
                    day[index]=day[index-1]
        records[day_index]=day
    new_dict[seg]=records 

In [ ]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充_padding.npy',new_dict)

In [77]:
new_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充_no_0.npy',allow_pickle=True).item()

In [78]:
for seg,records in new_dict.items():
    for day_index,day in enumerate(records):
        index=0
        new_day=[]
        while index<900:
            new_day.append(numpy.around(numpy.mean(day[index:index+5]),2))
            index+=5
        records[day_index]=new_day
    new_dict[seg]=records


In [80]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充_5min.npy',new_dict)

In [15]:
old_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\39_days_86seg_no_0.npy',allow_pickle=True).item()
# test.keys()

In [19]:
len(old_dict[5])

40

In [2]:
single_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\5-6月记录\0603\0603_raw_data.npy',allow_pickle=True).item()

In [3]:
new_single_dict={} # str->int,''->0
for seg,value in single_dict.items():
    day_record=list(value.values())
    for index in range(900):
        if day_record[index]=='':
            day_record[index]=0
        else:
            day_record[index]=int(day_record[index])
    new_single_dict[seg]=(day_record)

In [6]:
for seg,day in new_single_dict.items():#eliminate 0
    for index,each in enumerate(day):
        if each==0:
            if index==0:
                day[0]=min_tt[str(seg)]
            else:
                day[index]=day[index-1]
    new_single_dict[seg]=day 

In [7]:
new_single_arr=list(new_single_dict.values())
new_single_arr=numpy.array(new_single_arr)
numpy.sum(numpy.where(new_single_arr==0,1,0))

0

In [8]:
new_single_arr.shape

(422, 900)

In [18]:
for seg,old_records in old_dict.items():
    new_records=old_records[:19]+[new_single_dict[seg]]+old_records[19:]
    old_dict[seg]=new_records

In [20]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\40_days_86seg_no_0.npy',old_dict)

In [21]:
new_dict={}
for seg,day in new_single_dict.items(): # 5min slot in 0603 raw_data
    index=0
    new_day=[]
    while index<900:
        new_day.append(numpy.around(numpy.mean(day[index:index+5]),2))
        index+=5
    new_dict[seg]=new_day

In [23]:
old_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\39_days_86seg_no_0_5m.npy',allow_pickle=True).item()

In [24]:
for seg,old_records in old_dict.items():# add 0603
    new_records=old_records[:19]+[new_dict[seg]]+old_records[19:]
    old_dict[seg]=new_records

In [34]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\40_days_86seg_no_0_5m.npy',old_dict)

In [123]:
old_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\40天原始记录\40_days_86seg_no_0_5m.npy',allow_pickle=True).item()

In [81]:
new_dict=numpy.load(r'C:\Users\e-wel\Documents\Python Scripts\补充20天统计\补充_5min.npy',allow_pickle=True).item()

In [124]:
for seg,old_records in old_dict.items(): #0501-0507
    new_records=new_dict[seg][:7]+old_records
    old_dict[seg]=new_records

In [ ]:
len(old_dict[5]),len(old_dict[5][0]),len(new_dict[5]),len(new_dict[5][0])

In [125]:
for seg,old_records in old_dict.items():#add 0604
    new_records=old_records+[new_dict[seg][11]]
    old_dict[seg]=new_records

In [102]:
len(old_dict[5]),len(old_dict[5][0]),len(new_dict[5]),len(new_dict[5][0])

(48, 180, 20, 180)

In [126]:
len(old_dict[seg][-1])


180

In [127]:

for seg,old_records in old_dict.items():
    new_records=old_records[:12]+new_dict[seg][7:9] #'0513','0514'
    new_records+=old_records[12:17]+new_dict[seg][9:11] #'0520','0521'
    new_records+=old_records[17:22]+new_dict[seg][12:14] #'0527','0528'
    new_records+=old_records[22:33]+new_dict[seg][14:16] #'0610','0611'
    new_records+=old_records[33:38]+new_dict[seg][16:18] #'0617','0618'
    new_records+=old_records[38:43]+new_dict[seg][18:20] #'0624','0625'
    new_records+=old_records[43:48]
    old_dict[seg]=new_records

In [106]:
len(old_dict[9]),len(old_dict[9][0]),len(new_dict[5]),len(new_dict[5][0])

(60, 180, 20, 180)

In [105]:
old_dict.keys()

dict_keys([9, 24, 39, 40, 41, 42, 43, 44, 45, 47, 60, 83, 106, 107, 111, 112, 117, 118, 124, 138, 139, 150, 152, 153, 167, 168, 185, 186, 192, 193, 212, 213, 214, 224, 228, 246, 256, 267, 268, 272, 277, 316, 385, 1, 2, 3, 5, 23, 32, 33, 34, 37, 38, 46, 51, 52, 53, 54, 55, 56, 57, 58, 59, 71, 72, 101, 104, 137, 148, 166, 178, 187, 188, 217, 220, 221, 225, 257, 260, 264, 265, 297, 299, 300, 311, 314])

In [128]:
for seg,value in old_dict.items():#检查合并后的维度
    for index,each in enumerate(value):
        try:
            if len(each)!=180:
                print(seg,index)
        except:
            print(seg,index)

In [129]:
old_arr=list(old_dict.values())
old_arr=numpy.array(old_arr)
# numpy.sum(numpy.where(old_arr==0,1,0))
old_arr.shape

(86, 60, 180)

In [130]:
numpy.save(r'C:\Users\e-wel\Documents\Python Scripts\60天统计\60天_5min.npy',old_dict)

In [131]:
numpy.sum(numpy.where(old_arr==0,1,0))

0